<a href="https://colab.research.google.com/github/siperolli/IBM-Data-Science-Capstone-/blob/main/capstone_week_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>This notebook is part of
 week 3 assignment of the capstone project.

It consists of grouping the neighborhoods of Toronto. The neighborhood chosen as a reference is the most famous neighborhood in toronto Yorkville.

In [1]:
import numpy as np # library to handle data in a vectorized manner
import requests
import urllib
from bs4 import BeautifulSoup
import re

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Get data from the toronto neighborhood:

https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [3]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
r = requests.get(url)
# convert url em text
data = r.text
#convert 
soup = BeautifulSoup(data, 'html.parser')

Create a Dataframe= tb with the data os url

In [4]:
tb=soup.find('table').text


In [5]:
tb

"\n\n\nM1ANot assigned\n\n\n\nM2ANot assigned\n\n\n\nM3ANorth York(Parkwoods)\n\n\n\nM4ANorth York(Victoria Village)\n\n\n\nM5ADowntown Toronto(Regent Park / Harbourfront)\n\n\n\nM6ANorth York(Lawrence Manor / Lawrence Heights)\n\n\n\nM7AQueen's Park(Ontario Provincial Government)\n\n\n\nM8ANot assigned\n\n\n\nM9AEtobicoke(Islington Avenue)\n\n\n\n\nM1BScarborough(Malvern / Rouge)\n\n\n\nM2BNot assigned\n\n\n\nM3BNorth York(Don Mills)North\n\n\n\nM4BEast York(Parkview Hill / Woodbine Gardens)\n\n\n\nM5BDowntown Toronto(Garden District, Ryerson)\n\n\n\nM6BNorth York(Glencairn)\n\n\n\nM7BNot assigned\n\n\n\nM8BNot assigned\n\n\n\nM9BEtobicoke(West Deane Park / Princess Gardens / Martin Grove / Islington / Cloverdale)\n\n\n\n\nM1CScarborough(Rouge Hill / Port Union / Highland Creek)\n\n\n\nM2CNot assigned\n\n\n\nM3CNorth York(Don Mills)South(Flemingdon Park)\n\n\n\nM4CEast York(Woodbine Heights)\n\n\n\nM5CDowntown Toronto(St. James Town)\n\n\n\nM6CYork(Humewood-Cedarvale)\n\n\n\nM7CNot as

Separating only what we want:

In [ ]:
re.split('\n{3,5}', tb)

In [8]:
tab1=pd.DataFrame(re.split('\n{3,5}', tb))

In [ ]:
tab1.head(20)

Create a Dataframe

In [10]:
Toronto = pd.DataFrame()

Create column Postalcode with the part of the tb1.

In [11]:
Toronto['Postalcode']= tab1[0].str[0:3]

In [12]:
Toronto.shape

(181, 1)

Create column Borough with the part of the tb1.

In [14]:
Toronto['Borough']=tab1[0].str.extract('...(.*)\(')

In [ ]:
Toronto['Borough']

View Toronto

In [ ]:
Toronto

Test the function to extract another column: Neighborhood

In [ ]:
tab1[0].str.extract('...(.*)\(')

Create column Neighborhood with the part of the tb1.

In [18]:
Toronto['Neighborhood'] = tab1[0].str.extract('\((.*)\)')

In [ ]:
Toronto.head(50)

In [20]:
Toronto.shape

(181, 3)

Drop null values.

In [21]:
Toronto.dropna(thresh=2, inplace=True)

In [22]:
Toronto.shape

(103, 3)

Placing each neighborhood on a separate line

In [24]:
Neighborhood = Toronto.Neighborhood.str.split('/|,', expand=True).values.ravel()
Borough = np.repeat(Toronto.Borough.values, len(Neighborhood) / len(Toronto))
Postalcode = np.repeat(Toronto.Postalcode.values, len(Neighborhood) / len(Toronto))

Toronto = pd.DataFrame({'Postalcode': Postalcode,
              'Borough': Borough,
              'Neighborhood':Neighborhood})

Drop null values

In [25]:
Toronto.dropna(subset=['Neighborhood'], inplace=True)

In [26]:
Toronto.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
9,M4A,North York,Victoria Village
18,M5A,Downtown Toronto,Regent Park
19,M5A,Downtown Toronto,Harbourfront
27,M6A,North York,Lawrence Manor


Reset the index

In [27]:
Toronto.reset_index(drop=True)

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park
3,M5A,Downtown Toronto,Harbourfront
4,M6A,North York,Lawrence Manor
5,M6A,North York,Lawrence Heights
6,M7A,Queen's Park,Ontario Provincial Government
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Malvern
9,M1B,Scarborough,Rouge


In [29]:
Toronto.head(150)

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park
3,M5A,Downtown Toronto,Harbourfront
4,M6A,North York,Lawrence Manor
5,M6A,North York,Lawrence Heights
6,M7A,Queen's Park,Ontario Provincial Government
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Malvern
9,M1B,Scarborough,Rouge



The replaces below were done, because they were giving error in the *for*. As errors were found in the for, I changed the names of Borough or Neighborhood.


In [30]:
Toronto.replace({'MississaugaCanada Post Gateway Processing Centre':'Mississauga',
                 'Downtown TorontoStn A PO Boxes25 The Esplanade': 'Downtown',
                 'EtobicokeNorthwest':'Etobicoke Northwest',
                 'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business'},
                inplace=True)

Create latitude e longitude

In [31]:
!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

In [32]:
Toronto['Latitude'] = np.nan
Toronto['Longitude'] = np.nan



      Note:First mistake in the borough was: Ontario Provincial Government which is the Ontario Legislative Building at Quen's Park, so he went to Neighborhood Queen's Park to define Lat and Long.

In [33]:
for i in range(len(Toronto)):
  address = 'Toronto, '+ Toronto.iloc[i,2]
  geolocator = Nominatim(user_agent="foursquare_agent")
  location = geolocator.geocode(address)
  try: location.latitude
  except AttributeError as error:
    address = 'Toronto, '+ Toronto.iloc[i,1]
    location = geolocator.geocode(address)
    
  latitude = location.latitude
  longitude = location.longitude
  Toronto.iloc[i,3] = latitude
  Toronto.iloc[i,4] = longitude

In [39]:
Toronto.head(130)

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.758800,-79.320197
1,M4A,North York,Victoria Village,43.732658,-79.311189
2,M5A,Downtown Toronto,Regent Park,43.660706,-79.360457
3,M5A,Downtown Toronto,Harbourfront,43.640080,-79.380150
4,M6A,North York,Lawrence Manor,43.722079,-79.437507
5,M6A,North York,Lawrence Heights,43.722778,-79.450933
6,M7A,Queen's Park,Ontario Provincial Government,43.659659,-79.390340
7,M9A,Etobicoke,Islington Avenue,43.638959,-79.521050
8,M1B,Scarborough,Malvern,43.809196,-79.221701
9,M1B,Scarborough,Rouge,43.804930,-79.165837



Create a map of Toronto with neighborhoods superimposed on top.

In [35]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(Toronto['Latitude'], Toronto['Longitude'], Toronto['Borough'], Toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

In [ ]:
map_toronto

However, for illustration purposes, let's simplify the above map and segment and cluster only the neighborhoods in Yorkville. 

Is the most famous neighborhood in Toronto.

In [56]:
Yorkville_lat = Toronto.loc[129,'Latitude']
Yorkville_long = Toronto.loc[129,'Longitude']
print(Yorkville_lat, Yorkville_long)

43.6713861 -79.3901677


Define Foursquare Credentials and Version

In [37]:
CLIENT_ID = 'KRYP05IZOX5R5AVHGFDH1RE53PIVLGT3TRL3DH0Z1EN01R2B' # your Foursquare ID
CLIENT_SECRET = '5Q3K102L002VIIMRSBCR0VC3GS1QVACBHI0TFWAWAVQMFWUU' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KRYP05IZOX5R5AVHGFDH1RE53PIVLGT3TRL3DH0Z1EN01R2B
CLIENT_SECRET:5Q3K102L002VIIMRSBCR0VC3GS1QVACBHI0TFWAWAVQMFWUU


Now, let's get the top 100 venues that are in Yorkville within a radius of 500 meters.

In [57]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
 # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    Yorkville_lat,
    Yorkville_long, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=KRYP05IZOX5R5AVHGFDH1RE53PIVLGT3TRL3DH0Z1EN01R2B&client_secret=5Q3K102L002VIIMRSBCR0VC3GS1QVACBHI0TFWAWAVQMFWUU&v=20180605&ll=43.6713861,-79.3901677&radius=500&limit=100'

Send the GET request and examine the resutls

In [ ]:
results = requests.get(url).json()
results

From the Foursquare lab in the previous module, we know that all the information is in the items key. Before we proceed, let's borrow the get_category_type function from the Foursquare lab.

In [59]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a pandas dataframe.

In [60]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Paramount Fine Foods,Middle Eastern Restaurant,43.670677,-79.389865
1,Four Seasons Hotel Toronto,Hotel,43.671796,-79.389457
2,Fiorio Hair Design,Cosmetics Shop,43.670170,-79.391400
3,Bloor-Yorkville,Shopping Mall,43.670214,-79.391047
4,Yamato Japanese Restaurant,Japanese Restaurant,43.670683,-79.391056


And how many venues were returned by Foursquare?

In [61]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


2. Explore Neighborhoods in yorkville

Let's create a function to repeat the same process to all the neighborhoods in yorkville

In [63]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now write the code to run the above function on each neighborhood and create a new dataframe called yorkville_venues

In [66]:
# type your answer here
yorkville_venues = getNearbyVenues(names=Toronto['Neighborhood'],
                                   latitudes=Toronto['Latitude'],
                                   longitudes=Toronto['Longitude'])

Parkwoods
Victoria Village
Regent Park 
 Harbourfront
Lawrence Manor 
 Lawrence Heights
Ontario Provincial Government
Islington Avenue
Malvern 
 Rouge
Don Mills
Parkview Hill 
 Woodbine Gardens
Garden District
 Ryerson
Glencairn
West Deane Park 
 Princess Gardens 
 Martin Grove 
 Islington 
 Cloverdale
Rouge Hill 
 Port Union 
 Highland Creek
Don Mills)South(Flemingdon Park
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate 
 Bloordale Gardens 
 Old Burnhamthorpe 
 Markland Wood
Guildwood 
 Morningside 
 West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor 
 Wilson Heights 
 Downsview North
Thorncliffe Park
Richmond 
 Adelaide 
 King
Dufferin 
 Dovercourt Village
Scarborough Village
Fairview 
 Henry Farm 
 Oriole
Northwood Park 
 York University
The Danforth  East
Harbourfront East 
 Union Station 
 Toronto Islands
Little Portugal 
 Trinity
Kennedy Park 
 Ionview 
 East Birchmount Park
Bayvi

Let's check the size of the resulting dataframe

In [67]:
print(yorkville_venues.shape)
yorkville_venues.head()

(5876, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.7588,-79.320197,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant
1,Parkwoods,43.7588,-79.320197,LCBO,43.757774,-79.314257,Liquor Store
2,Parkwoods,43.7588,-79.320197,Shoppers Drug Mart,43.760857,-79.324961,Pharmacy
3,Parkwoods,43.7588,-79.320197,Petro-Canada,43.757950,-79.315187,Gas Station
4,Parkwoods,43.7588,-79.320197,Pizza Pizza,43.760231,-79.325666,Pizza Place


Let's check how many venues were returned for each neighborhood

In [ ]:
yorkville_venues.groupby('Neighborhood').count()

Let's find out how many unique categories can be curated from all the returned venues

In [69]:
print('There are {} uniques categories.'.format(len(yorkville_venues['Venue Category'].unique())))

There are 337 uniques categories.


3. Analyze Each Neighborhood

In [71]:
# one hot encoding
yorkville_onehot = pd.get_dummies(yorkville_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
yorkville_onehot['Neighborhood'] = yorkville_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [yorkville_onehot.columns[-1]] + list(yorkville_onehot.columns[:-1])
yorkville_onehot = yorkville_onehot[fixed_columns]

yorkville_onehot.head()

,Yoga Studio,ATM,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Service,Airport Terminal,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Trail,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cable Car,Café,Cajun / Creole Restaurant,Camera Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Circus,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Doctor's Office,Dog Run,Doner Restaurant,Dongbei Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Service,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Housing Development,Hungarian Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Laser Tag,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Library,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Moving Target,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Night Market,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Supply Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Peking Duck Restaurant,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Physical Therapist,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Post Office,Poutine Place,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Climbing 

And let's examine the new dataframe size.

In [72]:
yorkville_onehot.shape

(5876, 337)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [ ]:
yorkville_grouped = yorkville_onehot.groupby('Neighborhood').mean().reset_index()
yorkville_grouped

In [74]:
yorkville_grouped.shape

(210, 337)

Let's print each neighborhood along with the top 5 most common venues

In [75]:
num_top_venues = 5

for hood in yorkville_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = yorkville_grouped[yorkville_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

---- Adelaide ----
                 venue  freq
0          Coffee Shop  0.07
1                 Café  0.06
2  Japanese Restaurant  0.05
3   Italian Restaurant  0.05
4                  Gym  0.05


---- Agincourt North ----
                 venue  freq
0                 Bank  0.08
1   Chinese Restaurant  0.08
2           Beer Store  0.04
3  Sporting Goods Shop  0.04
4                  Spa  0.04


---- Albion Gardens----
                 venue  freq
0        Grocery Store  0.18
1       Sandwich Place  0.09
2  Fried Chicken Joint  0.09
3          Pizza Place  0.09
4             Pharmacy  0.09


---- Bathurst Quay ----
           venue  freq
0    Coffee Shop  0.16
1           Café  0.12
2  Grocery Store  0.08
3           Park  0.08
4   Dance Studio  0.04


---- Beaumond Heights ----
               venue  freq
0  Convenience Store  0.09
1      Grocery Store  0.09
2         Restaurant  0.09
3     Clothing Store  0.09
4        Coffee Shop  0.09


---- Bloordale Gardens ----
               venue

Let's put that into a pandas dataframe

First, let's write a function to sort the venues in descending order.

In [76]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [78]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = yorkville_grouped['Neighborhood']

for ind in np.arange(yorkville_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(yorkville_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Café,Gym,Italian Restaurant,Japanese Restaurant,Hotel,Restaurant,Gastropub,Asian Restaurant,American Restaurant
1,Agincourt North,Bank,Chinese Restaurant,Convenience Store,Beer Store,Clothing Store,Japanese Restaurant,Juice Bar,Restaurant,Movie Theater,Discount Store
2,Albion Gardens,Grocery Store,Pharmacy,Liquor Store,Auto Garage,Sandwich Place,Caribbean Restaurant,Fast Food Restaurant,Beer Store,Fried Chicken Joint,Pizza Place
3,Bathurst Quay,Coffee Shop,Café,Grocery Store,Park,Sushi Restaurant,Tunnel,Garden,Harbor / Marina,Diner,Sculpture Garden
4,Beaumond Heights,Hotel,Transportation Service,Coffee Shop,Clothing Store,Restaurant,Café,Farmers Market,Skating Rink,Grocery Store,Intersection


4. Cluster Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [95]:
# set number of clusters
kclusters = 5

yorkville_grouped_clustering = yorkville_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(yorkville_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:100] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 0, 0, 1, 2, 1, 0, 2, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0,
       4, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 3, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [82]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

yorkville_merged = Toronto

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
yorkville_merged = yorkville_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

yorkville_merged.head() # check the last columns!

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.758800,-79.320197,1.0,Chinese Restaurant,Bus Line,Electronics Store,Laundry Service,Gas Station,Coffee Shop,Discount Store,Bank,Liquor Store,Caribbean Restaurant
1,M4A,North York,Victoria Village,43.732658,-79.311189,1.0,Mediterranean Restaurant,Middle Eastern Restaurant,Asian Restaurant,Thai Restaurant,Falafel Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Elementary School,Escape Room
2,M5A,Downtown Toronto,Regent Park,43.660706,-79.360457,1.0,Coffee Shop,Restaurant,Thai Restaurant,Indian Restaurant,Sushi Restaurant,Pub,Beer Store,Auto Dealership,Grocery Store,Electronics Store
3,M5A,Downtown Toronto,Harbourfront,43.640080,-79.380150,1.0,Coffee Shop,Café,Hotel,Restaurant,Pizza Place,Italian Restaurant,Steakhouse,Park,Chinese Restaurant,Plaza
4,M6A,North York,Lawrence Manor,43.722079,-79.437507,0.0,Kids Store,Bank,Electronics Store,Doctor's Office,Park,Falafel Restaurant,Eastern European Restaurant,Egyptian Restaurant,Elementary School,Escape Room


Finally, let's visualize the resulting clusters

In [85]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(yorkville_merged['Latitude'], yorkville_merged['Longitude'], yorkville_merged['Neighborhood'], yorkville_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow,
        fill=True,
        fill_color=rainbow,
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

5. Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

Cluster 1

In [90]:
yorkville_merged.loc[yorkville_merged['Cluster Labels'] == 0, yorkville_merged.columns[[1] + list(range(5, yorkville_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,North York,0.0,Kids Store,Bank,Electronics Store,Doctor's Office,Park,Falafel Restaurant,Eastern European Restaurant,Egyptian Restaurant,Elementary School,Escape Room
9,Scarborough,0.0,Caribbean Restaurant,Fast Food Restaurant,Park,Food & Drink Shop,Flower Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Elementary School,Escape Room
12,East York,0.0,Coffee Shop,Park,Bakery,Home Service,Falafel Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Elementary School,Escape Room
18,Etobicoke,0.0,Greek Restaurant,Supermarket,Gym,Athletics & Sports,Park,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Elementary School,Escape Room
23,Scarborough,0.0,Park,Pharmacy,IT Services,Farmers Market,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant
25,East York,0.0,Skating Rink,Pharmacy,Dance Studio,Park,Athletics & Sports,Bus Stop,Intersection,Women's Store,Falafel Restaurant,Electronics Store
30,Etobicoke,0.0,Dog Run,Park,Flower Shop,Gas Station,Falafel Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Elementary School,Escape Room
31,Etobicoke,0.0,Golf Course,Piano Bar,Baseball Field,Park,Falafel Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Elementary School,Escape Room
33,Scarborough,0.0,Coffee Shop,Park,Café,Sandwich Place,Discount Store,Supermarket,Mobile Phone Shop,Convenience Store,Fast Food Restaurant,Pharmacy
39,East York,0.0,Convenience Store,Sandwich Place,Japanese Restaurant,Park,Women's Store,Event Space,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Elementary School


Cluster 2

In [ ]:
yorkville_merged.loc[yorkville_merged['Cluster Labels'] == 1, yorkville_merged.columns[[1] + list(range(5, yorkville_merged.shape[1]))]]

Cluster 3

In [93]:
yorkville_merged.loc[yorkville_merged['Cluster Labels'] == 2, yorkville_merged.columns[[1] + list(range(5, yorkville_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
132,Mississauga,2.0,Airport,Airport Terminal,Women's Store,Fast Food Restaurant,Egyptian Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space
166,Downtown Toronto,2.0,Airport,Airport Terminal,Women's Store,Fast Food Restaurant,Egyptian Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space
172,Etobicoke,2.0,Airport,Airport Terminal,Women's Store,Fast Food Restaurant,Egyptian Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space
184,Etobicoke Northwest,2.0,Airport,Airport Terminal,Women's Store,Fast Food Restaurant,Egyptian Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space
187,Etobicoke Northwest,2.0,Airport,Airport Terminal,Women's Store,Fast Food Restaurant,Egyptian Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space


Cluster 4

In [92]:
yorkville_merged.loc[yorkville_merged['Cluster Labels'] == 3, yorkville_merged.columns[[1] + list(range(5, yorkville_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
153,Scarborough,3.0,Playground,Women's Store,Farmers Market,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space
170,Etobicoke,3.0,Playground,Women's Store,Farmers Market,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space
178,Scarborough,3.0,Playground,Women's Store,Farmers Market,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space


Cluster 5

In [91]:
yorkville_merged.loc[yorkville_merged['Cluster Labels'] == 4, yorkville_merged.columns[[1] + list(range(5, yorkville_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Scarborough,4.0,Park,Women's Store,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant
28,Etobicoke,4.0,Park,Electronics Store,Women's Store,Eastern European Restaurant,Egyptian Restaurant,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant
57,North York,4.0,Park,Construction & Landscaping,Baseball Field,Farmers Market,Egyptian Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space
81,North York,4.0,Park,Middle Eastern Restaurant,Fast Food Restaurant,Egyptian Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant
133,Etobicoke,4.0,Park,Women's Store,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant
180,Downtown Toronto,4.0,Park,Playground,Bike Trail,Women's Store,Farmers Market,Egyptian Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant
